# 餐飲業逾期風險預測模型
由我國「應建立食品追溯追蹤系統之食品業者」規定可知，須申報食品流向資料於本署非追不可系統多為製造與輸入業者，對於販售與餐飲業者則無相關資料可用，無法推算其庫存，進而評估其是否可能存放逾期食品。因此，針對販售與餐飲業者，則透過後市場稽查資料及業者相關資訊，嘗試以監督式學習方法建構業者逾期食品違規風險預測模型，而此處為餐飲業資料的建模評估。

### 程式架構
* 資料前處理(Preprocessing.ipynb)
* 特徵篩選(FeatureSelection.ipynb)
* 資料不平衡處理(HandlingImbData.ipynb)
* **模型配適(ModelFitting.ipynb)**
* 模型評估(ModelEvaluating.ipynb)

## 模型配適

### Package

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data Import

In [2]:
dict_dfs_sm = pd.read_pickle('../data/data_sm.pkl')
print(dict_dfs_sm.keys())

dict_keys(['others', 'train', 'test', 'feature_selection'])


### Data Transform

In [3]:
X_train = dict_dfs_sm['train']['X_select']    # fit transformer
X_smote = dict_dfs_sm['train']['X_smote']
X_test = dict_dfs_sm['test']['X_select']

##### Encoding

In [4]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer

In [5]:
cols_DumEnc = [c for c in X_smote.select_dtypes('category').columns if not X_smote[c].dtypes.ordered]
cols_OrdEnc = [c for c in X_smote.select_dtypes('category').columns if X_smote[c].dtypes.ordered]

trans_enc = make_column_transformer(
    (OneHotEncoder(drop='first', sparse_output=False, dtype='uint8'), cols_DumEnc), 
    (OrdinalEncoder(dtype='uint8'), cols_OrdEnc), 
    remainder='passthrough', 
    force_int_remainder_cols=False, 
    verbose_feature_names_out=False
).set_output(transform='pandas')

# fit
trans_enc.fit(X_train)

# transform
X_train_Enc = trans_enc.transform(X_train)
X_sm_Enc = trans_enc.transform(X_smote)
X_test_Enc = trans_enc.transform(X_test)

##### Standardization

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
# scaler & scaler column names
scaler_stand = StandardScaler().set_output(transform='pandas')
cols_stand = X_smote.select_dtypes(include='float').columns.tolist() + cols_OrdEnc

# fit (use original training set)
scaler_stand.fit(X_train_Enc[cols_stand])

# transform (and create new df)
def scaler_df(df, cols=cols_stand, scaler=scaler_stand): 
    df_scale = scaler.transform(df[cols])
    df_all = pd.concat([df_scale, df.drop(cols, axis=1)], axis=1)
    return df_all

X_sm_EncSca = scaler_df(X_sm_Enc)
X_test_EncSca = scaler_df(X_test_Enc)

In [8]:
del cols_DumEnc, cols_OrdEnc, cols_stand, trans_enc, scaler_stand, X_train, X_train_Enc, X_smote, X_sm_Enc, X_test, X_test_Enc

### Shuffle

In [9]:
from sklearn.utils import shuffle
# maybe for bagging

In [10]:
y_smote = dict_dfs_sm['train']['y_smote']
y_sh, X_sh = shuffle(y_smote, X_sm_EncSca, random_state=27)

In [11]:
del X_sm_EncSca, y_smote

### Model Fitting: Logistic Regression (with L1 penalty) 

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
# create a model and fit the dataset
def fit_logistic_l1(X, y): 
    mdl = LogisticRegression(penalty='l1', solver='liblinear', max_iter=100, random_state=27)
    mdl.fit(X, y)
    return mdl

logistic_l1 = fit_logistic_l1(X_sh, y_sh)

In [14]:
# 迴歸係數
pd.DataFrame(
    {'column name': X_sh.columns.tolist(), 
     'coefficient': logistic_l1.coef_.flatten()}  
)

,column name,coefficient
0,資本額,0.985436
1,餐飲業次業別數,-1.472191
2,標章分數,-0.385332
3,場所規模,-0.542008
4,營業縣市_台中市,-1.594998
5,營業縣市_台北市,-0.427375
6,營業縣市_台南市,4.859250
7,營業縣市_台東縣,6.722968
8,營業縣市_嘉義市,3.409126
9,營業縣市_嘉義縣,0.000000


### Save

In [15]:
dict_mdl = {
    'logistic_l1': logistic_l1
}

In [16]:
import pickle

with open('../data/model.pkl', 'wb') as f: 
    pickle.dump(dict_mdl, f)

In [17]:
dict_dfs_sm['test']['X_stand'] = X_test_EncSca
pd.to_pickle(dict_dfs_sm, '../data/data_trans.pkl')